In [7]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [4]:
# 读取原始数据
data_origin = pd.read_csv("[new] yancheng_train_20171226.csv")
# 读取训练集
train_origin = pd.read_csv("train_origin.csv")

In [11]:
# 构造四个验证集
# 构造第1个验证集
cv1_train_X = train_origin[(train_origin["label"] >= 0) &
                          (train_origin["sale_date"] < 69)].copy()
cv1_train_y = cv1_train_X["label"]
cv1_test_X = train_origin[(train_origin["label"] >= 0) & (train_origin["sale_date"] == 69)].copy()
cv1_test_y = cv1_test_X["label"]
# 构造第2个验证集
cv2_train_X = train_origin[(train_origin["label"] >= 0) &
                          (train_origin["sale_date"] < 70) & (train_origin["sale_date"] != 58)].copy()
cv2_train_y = cv2_train_X["label"]
cv2_test_X = train_origin[(train_origin["label"] >= 0) & (train_origin["sale_date"] == 58)].copy()
cv2_test_y = cv2_test_X["label"]
# 构造第3个验证集
cv3_train_X = train_origin[(train_origin["label"] >= 0) &
                          (train_origin["sale_date"] < 69) & (train_origin["sale_date"] != 58)].copy()
cv3_train_y = cv3_train_X["label"]
cv3_test_X = train_origin[(train_origin["label"] >= 0) &
                          (train_origin["sale_date"] == 69)].copy()
cv3_test_y = cv3_test_X["label"]
# 构造第4个验证集
cv4_train_X = train_origin[(train_origin["label"] >= 0) &
                          (train_origin["sale_date"] < 69) & (train_origin["sale_date"] != 58)].copy()
cv4_train_y = cv4_train_X["label"]
cv4_test_X = train_origin[(train_origin["label"] >= 0) &
                          (train_origin["sale_date"] == 58)].copy()
cv4_test_y = cv4_test_X["label"]

In [13]:
# 训练各验证集，构造同时包含真实值和预测值的DataFrame
xgb_param = {'max_depth':4, 'eta':0.02, 'silent':1, 'objective':'reg:linear', "subsample": 0.8, "num_round": 1000,
             "early_stopping_rounds": 100}
result_df_lst = []
def cross_validation(feature_tmp):
    cv_train_X_lst = [cv1_train_X, cv2_train_X, cv3_train_X, cv4_train_X]
    cv_train_y_lst = [cv1_train_y, cv2_train_y, cv3_train_y, cv4_train_y]
    cv_test_X_lst = [cv1_test_X, cv2_test_X, cv3_test_X, cv4_test_X]
    cv_test_y_lst = [cv1_test_y, cv2_test_y, cv3_test_y, cv4_test_y]
    error_lst = []
    for i in range(len(cv_train_X_lst)):
        print "========第 %d 次交叉验证==========" % i
        dtrain_cv = xgb.DMatrix(cv_train_X_lst[i][feature_tmp].values, cv_train_y_lst[i])
        dtest_cv = xgb.DMatrix(cv_test_X_lst[i][feature_tmp].values, cv_test_y_lst[i])
        watchlist_cv = [(dtest_cv, "test_cv")]
        model_cv = xgb.train(xgb_param, dtrain_cv, xgb_param['num_round'], watchlist_cv, verbose_eval=10)
        result = np.array(model_cv.predict(dtest_cv))
        error_lst.append(np.sqrt(np.mean(np.power(result - cv_test_y_lst[i], 2))))
        result_df = cv_test_X_lst[i][["sale_date", "class_id"]].copy()
        result_df["predict"] = result
        result_df["true_value"] = cv_test_y_lst[i]
        result_df_lst.append(result_df)
    return error_lst


feature_tmp = ["sales", "car_length_label_mean", "if_charging_count", "rated_passenger_mean", "cylinder_number_mean",
               "TR_count", "min_price_mean", "fuel_type_id_count", "driven_type_id_count", "gearbox_type_count",
               "newenergy_type_id_count", "min_price_label_mean", "car_height_mean", "engine_torque_mean",
               "rated_passenger_label_mean", "displacement_count", "displacement_mean", "max_price_count",
               "car_length_mean", "rated_passenger_count", "max_price_median", "min_price_median", "avg_price_median",
               "sales_is_increase", "is_price_smaller_min_price", "is_price_in_price_level",
               'type_id_1_label_sum','type_id_3_label_sum','wheelbase_history_mean', 'front_track_history_mean',
               'max_price_history_mean', 'min_price_history_mean', 'TR_history_count','gearbox_type_history_count',
               'displacement_history_count','if_charging_history_count','driven_type_id_history_count',
               'fuel_type_id_history_count','newenergy_type_id_history_count','rated_passenger_history_count',
               'max_price_history_count']
error_lst = cross_validation(feature_tmp=feature_tmp)
print error_lst

========第 0 次交叉验证==========
[0]	test_cv-rmse:565.727
[10]	test_cv-rmse:463.488
[20]	test_cv-rmse:381.88
[30]	test_cv-rmse:316.98
[40]	test_cv-rmse:266.052
[50]	test_cv-rmse:227.03
[60]	test_cv-rmse:196.568
[70]	test_cv-rmse:175.267
[80]	test_cv-rmse:162.168
[90]	test_cv-rmse:152.359
[100]	test_cv-rmse:145.837
[110]	test_cv-rmse:142.384
[120]	test_cv-rmse:140.089
[130]	test_cv-rmse:139.008
[140]	test_cv-rmse:137.323
[150]	test_cv-rmse:136.477
[160]	test_cv-rmse:137.44
[170]	test_cv-rmse:137.373
[180]	test_cv-rmse:137.808
[190]	test_cv-rmse:138.328
[200]	test_cv-rmse:138.719
[210]	test_cv-rmse:138.687
[220]	test_cv-rmse:139.329
[230]	test_cv-rmse:139.833
[240]	test_cv-rmse:139.822
[250]	test_cv-rmse:140.187
[260]	test_cv-rmse:140.511
[270]	test_cv-rmse:140.068
[280]	test_cv-rmse:140.493
[290]	test_cv-rmse:139.613
[300]	test_cv-rmse:138.547
[310]	test_cv-rmse:138.616
[320]	test_cv-rmse:138.329
[330]	test_cv-rmse:137.882
[340]	test_cv-rmse:137.447
[350]	test_cv-rmse:137.435
[360]	test_cv-r

========第 3 次交叉验证==========
[0]	test_cv-rmse:824.717
[10]	test_cv-rmse:702.084
[20]	test_cv-rmse:602.062
[30]	test_cv-rmse:517.571
[40]	test_cv-rmse:450.238
[50]	test_cv-rmse:394.023
[60]	test_cv-rmse:349.031
[70]	test_cv-rmse:310.439
[80]	test_cv-rmse:281.589
[90]	test_cv-rmse:256.874
[100]	test_cv-rmse:239.422
[110]	test_cv-rmse:223.641
[120]	test_cv-rmse:212.609
[130]	test_cv-rmse:203.441
[140]	test_cv-rmse:194.953
[150]	test_cv-rmse:189.907
[160]	test_cv-rmse:186.097
[170]	test_cv-rmse:180.992
[180]	test_cv-rmse:177.419
[190]	test_cv-rmse:176.246
[200]	test_cv-rmse:174.068
[210]	test_cv-rmse:173.026
[220]	test_cv-rmse:172.511
[230]	test_cv-rmse:171.882
[240]	test_cv-rmse:170.734
[250]	test_cv-rmse:169.233
[260]	test_cv-rmse:168.727
[270]	test_cv-rmse:167.337
[280]	test_cv-rmse:165.79
[290]	test_cv-rmse:165.408
[300]	test_cv-rmse:163.722
[310]	test_cv-rmse:163.191
[320]	test_cv-rmse:162.462
[330]	test_cv-rmse:162.369
[340]	test_cv-rmse:162.596
[350]	test_cv-rmse:162.87
[360]	test_cv

In [19]:
result_df0 = result_df_lst[0]
result_df3 = result_df_lst[3]
class_id1 = result_df0[np.abs(result_df0["predict"] - result_df0["true_value"]) > 135.018679][["class_id"]]
class_id2 = result_df3[np.abs(result_df3["predict"] - result_df3["true_value"]) > 161.161706][["class_id"]]
print pd.merge(class_id1, class_id2, on=["class_id"], how="inner")

   class_id
0    206765
1    354068
2    654134
3    735971
4    914348
5    924154


In [23]:
class_id_merge = pd.merge(class_id1, class_id2, on=["class_id"], how="inner")
print pd.merge(result_df0, class_id_merge, on="class_id", how="inner")
print pd.merge(result_df3, class_id_merge, on="class_id", how="inner")

   sale_date  class_id      predict  true_value
0         69    206765  1703.912476        2057
1         69    354068   916.611389         460
2         69    654134   732.091064         514
3         69    735971  2536.655029        2012
4         69    914348   623.666626         483
5         69    924154   985.558899        1192
   sale_date  class_id      predict  true_value
0         58    206765  1904.310059        1592
1         58    354068   843.344727        1077
2         58    654134   860.691833        1326
3         58    735971  2975.538818        3267
4         58    914348   768.414795        1024
5         58    924154  1462.340576        1274


In [25]:
print pd.merge(class_id_merge, cv1_train_X[cv1_train_X["sale_date"] == 68][["class_id", "label"]], 
         on=["class_id"], how="inner")
print pd.merge(class_id_merge, cv1_train_X[cv1_train_X["sale_date"] == 57][["class_id", "label"]], 
         on=["class_id"], how="inner")

   class_id  label
0    206765   2347
1    354068    920
2    654134    654
3    735971   2881
4    914348    650
5    924154   1010
   class_id  label
0    206765   1581
1    354068    736
2    654134    846
3    735971   2512
4    914348    680
5    924154   1302


In [28]:
data_origin[data_origin["class_id"] == 206765].to_csv("test.csv", index=None)

In [44]:
result_df0[np.abs(result_df0["predict"] - result_df0["true_value"]) > 100]

,sale_date,class_id,predict,true_value
5309,69,125403,313.186798,156
5315,69,194201,171.994949,378
5318,69,206765,1703.912476,2057
5319,69,209945,265.374725,91
5325,69,250658,307.253876,187
5326,69,265980,460.854309,204
5327,69,270690,844.456604,595
5328,69,281301,608.218506,329
5330,69,289386,297.515961,441
5332,69,290854,437.736786,205


In [39]:
result_df3[(np.abs(result_df3["predict"] - result_df3["true_value"]) > 100) & 
           (result_df3["predict"] < result_df3["true_value"])].size

88

In [40]:
result_df0 = result_df_lst[0]
result_df3 = result_df_lst[3]
class_id1 = result_df0[np.abs(result_df0["predict"] - result_df0["true_value"]) > 100][["class_id"]]
class_id2 = result_df3[np.abs(result_df3["predict"] - result_df3["true_value"]) > 100][["class_id"]]
print pd.merge(class_id1, class_id2, on=["class_id"], how="inner")

    class_id
0     206765
1     270690
2     281301
3     291086
4     347384
5     349023
6     354068
7     597927
8     621073
9     654134
10    714152
11    735971
12    854079
13    914348
14    924154


In [42]:
data_origin[data_origin["class_id"] == 206765].sort_values(["sale_date"]).to_csv("test.csv", index=None)